# Elements of Programming Interviews
## Linked Lists
### Track 4: 8.1, 8.2, 8.4, 8.5, 8.8, 8.11, 8.12

### 8.1 - Merge Two Sorted Lists
>Consider two singly linked lists in which each node holds a number. Assume the lists are sorted, i.e. numbers in the lists appear in ascending order within each list. The merge of two lists is a list consisting of the nods of the two lists in which numbers appear in ascending order. 
>>The only field you can change in a node is *next*.

In [1]:
import os
os.chdir("/home/william/Python/Algorithms/Data Structures")
from singly_linked_list import SinglyLinkedList

In [2]:
vals1 = [1, 2, 3, 6, 8, 10]
vals2 = [4, 5, 7, 8, 9, 55]
list1 = SinglyLinkedList()
list2 = SinglyLinkedList()
for val in vals1:
    list1.append(val)
for val in vals2:
    list2.append(val)

In [3]:
def merge_two_lists_bf(list1, list2):
    res = SinglyLinkedList()
    iter1, iter2 = list1.head, list2.head
    while iter1 and iter2:
        if iter1.data >= iter2.data:
            res.append(iter2)
            iter2 = iter2.next_
        else: #iter2.data > iter1.data
            res.append(iter1)
            iter1 = iter1.next_
    #append the last remaining elements of the valid list
    if iter1:
        res.append(iter1)
    else:
        res.append(iter2)
    return res

In [4]:
res = merge_two_lists_bf(list1, list2)
res.print_list()

1 2 3 4 5 6 7 8 8 9 10 55


### 8.2 - Reverse a Singly Linked List
>Give a linear time non-recursive function that reverses a singly linked list. The function should use no more than constant storage beyond that needed for the list itself.

In [5]:
def reverse_singly_list_1st(l):
    r_next_node, next_node, tmp = None, None, None
    iter_ = l.head
    
    while iter_ is not None:
        if iter_ is l.head:
            r_next_node = None
        else:
            r_next_node = tmp
        tmp = iter_
        next_node = iter_.next_
        iter_.next_ = r_next_node
        iter_ = next_node
        if next_node is None:
            l.head = tmp
    
def reverse_singly_list(l):
    curr = l.head
    prev = None
    while(curr):
        next_ = curr.next_
        curr.next_ = prev
        prev = curr
        curr = next_
        if next_ is None:
            l.head = prev

In [6]:
list1 = SinglyLinkedList()
for i in range(1,11):
    list1.append(i)
list1.print_list()
reverse_singly_list(list1)
print "\n"
list1.print_list()

1 2 3 4 5 6 7 8 9 10 

10 9 8 7 6 5 4 3 2 1


### 8.4 - Test for Cyclicity
>Given a reference to the head of a singly linked list how would you determine whether the list ends in a null or reaches a cycle of nodes. Write a program that returns null if no cycle is presend, or a reference to the start of the cycle if one is present. You do not know the length of the list in advance,

#### This implementation is bad on space complexity, as it uses O(N) space.

In [7]:
#bad space complexity
def test_list_for_cyclicity_bf(l):
    visited_nodes = []
    curr = l.head
    while curr not in visited_nodes and curr is not None:
        visited_nodes.append(curr)
        curr = curr.next_
    return curr

In [8]:
cyclic_list = SinglyLinkedList()
for i in range(1,8):
    cyclic_list.append(i)
#start cycle at second node
start_cycle_node = cyclic_list.head.next_
end_cycle_node = start_cycle_node.next_.next_.next_.next_
end_cycle_node.next_ = start_cycle_node

acyclic_list = SinglyLinkedList()
for i in range(1,11):
    acyclic_list.append(i)

In [9]:
print test_list_for_cyclicity_bf(cyclic_list) is start_cycle_node
print test_list_for_cyclicity_bf(acyclic_list) is None

True
True


>This implementation uses two iterators. In the outer loop, the slow iterator is advanced once per loop, whereas the fast iterator will advance twice. If there is a cycle, then eventually the fast iterator and the slow iterator will meet. 

>Once a cycle is found, it's length must be detrmined. 

>In order to find the beginning of the cycle, you must have the length, $C$, and must start a traversal from the beginning of the list. One iterator, $start$, will begin at the head, and the other iterator, $end$ will be $C$ nodes ahead of $start$. Both iterators will be advanced once, until $ends$ next_ node is $start$.

In [20]:
def get_cycle_length(l, slow_iter, fast_iter):
    fast_iter = fast_iter.next_
    length = 1
    while fast_iter.next_ is not slow_iter:
        fast_iter = fast_iter.next_
        length += 1
    return length
    
def test_list_for_cyclicity(l):
    fast_iter = l.head
    slow_iter = l.head
    while fast_iter and fast_iter.next_ and fast_iter.next_.next_:
        slow_iter = slow_iter.next_
        fast_iter = fast_iter.next_.next_
        if slow_iter is fast_iter:
            break
    else:
        #no cycle found
        return None
    #get cycle length
    length = 0
    length = get_cycle_length(l, slow_iter, fast_iter)
    start = l.head
    end = l.head
    for _ in range(length):
        end = end.next_
    while end.next_ is not start:
        start = start.next_
        end = end.next_
    return start.data

In [24]:
print test_list_for_cyclicity(list1)
test_list_for_cyclicity(cyclic_list)

None


2

### 8.5 - Test for Overlapping Lists -- Lists Are Cycle Free
>Write a program that takes two cycle-free singly linked lists and determines if there exists a node common to both lists.

>> My first initial approach is two fully traverse both lists, and if the last node of both lists is the same, then the two lists must be linked.

In [33]:
# Creating the overlapping lists
#olap_list1: [1]-->[2]-->[3]-->[4]----.
#                                     |
#olap_list2: [1]-->[2]-->[3]-->[4]-->[5]-->[6]-->[7]-->[8]-->[9]-->[10]
olap_list1 = SinglyLinkedList()
olap_list2 = SinglyLinkedList()
for num in range(1, 11):
    olap_list2.append(num)
for num in range(1, 5):
    olap_list1.append(num)
iter1 = olap_list1.head.next_.next_.next_
olap_iter = olap_list2.head.next_.next_.next_.next_
iter1.next_ = olap_iter

In [25]:
def test_for_overlap_bf(list1, list2):
    iter1 = list1.head
    iter2 = list2.head
    while iter1.next_ is not None:
        iter1 = iter1.next_
    while iter2.next_ is not None:
        iter2 = iter2.next_
    return iter1 is iter2

In [32]:
test_for_overlap_bf(olap_list1, olap_list2), test_for_overlap_bf(list1, list2)

(True, False)

>>The next approach is to simply start at the head of both lists, then advance both in tandem. If the iterators ever meet then there must be an overlap.

In [34]:
def test_for_overlap(list1, list2):
    iter1 = list1.head
    iter2 = list2.head
    while iter1 is not None and iter2 is not None:
        iter1 = iter1.next_
        iter2 = iter2.next_
        if iter1 is iter2:
            return True
    return False

In [35]:
test_for_overlap(olap_list1, olap_list2), test_for_overlap(list1, list2)

(True, False)

### 8.8 - Remove the $Kth$  Last Element From a List
>Given a singly linked list and an integer $k$, write a program to remove the $kth$ last element from the list. Your algorithm cannot use more than a few words of storage, regardless of the length of the list. In particular, you cannot assume that it is possible to record the length of the list.

>>This approach is trivial, but violates the rule that it is not possible to record the length of the list.

In [49]:
def find_kth_last_element_with_length(l, k):
    length = 0
    iter_ = l.head
    while iter_ is not None:
        length += 1
        iter_ = iter_.next_
    kth_last = l.head
    for _ in range((length - k)):
        kth_last = kth_last.next_
    return kth_last.data

In [50]:
#kth_list: [1]-->[2]-->[3]-->[4]-->[5]-->[6]-->[7]-->[8]-->[9]-->[10]
kth_list = SinglyLinkedList()
for num in range(1, 11):
    kth_list.append(num)

In [51]:
find_kth_last_element_with_length(kth_list, 2), find_kth_last_element(kth_list, 5)

(9, 6)

>>This problem becomes much easier if you use two iterators. The first iterator, $front$, should stay $k$ nodes behind the second iterator $back$. Then, advance both iterators in tandem, until the $back$ iterator is at the tail. At this point, the $front$ iterator will be on the $kth+1$ last element.

In [60]:
def remove_kth_last_element(l, k):
    front = l.head
    back = l.head
    #advance back k elements ahead of front
    for _ in range(k):
        back = back.next_
    #when the loop stops, back will be past the tail, and front will be kth_last
    while back.next_ is not None:
        front = front.next_
        back = back.next_
    front.next_ = front.next_.next_
    return l

In [63]:
#kth_list: [1]-->[2]-->[3]-->[4]-->[5]-->[6]-->[7]-->[8]-->[9]-->[10]
kth_list = SinglyLinkedList()
for num in range(1, 11):
    kth_list.append(num)
remove_kth_last_element(kth_list, 2).print_list()

1 2 3 4 5 6 7 8 10


### 8.11 - Implement Even-Odd Merge of a List
>Consider a singly linked list whose nodes are numbered starting at 0. Define the even-odd merge of the list to be the list consiting ofthe even-numbered nodes followed by the odd-numbered nodes.

Unmerged List := [$n_0$]-->[$n_1$]-->[$n_2$]-->[$n_3$]-->[$n_4$]-->[$n_5$]


Even-Odd-Merge: [$n_0$]-->[$n_2$]-->[$n_4$]-->[$n_1$]-->[$n_3$]-->[$n_5$]

In [64]:
def even_odd_merge(l):
    even = l.head
    odd = l.head.next_
    link_between = l.head.next_
    while odd.next_ is not None:
        even.next_ = odd.next_
        even = even.next_
        odd.next_ = even.next_
        odd = odd.next_
    even.next_ = link_between
    return l

In [66]:
eo_list = SinglyLinkedList()
for num in range(6):
    eo_list.append(num)
even_odd_merge(eo_list).print_list()

0 2 4 1 3 5
